In [317]:
using Pkg
Pkg.add("DataFrames")
Pkg.add("CSV")
Pkg.add("TickTock")

   Resolving package versions...
  No Changes to `/opt/julia/environments/v1.7/Project.toml`
  No Changes to `/opt/julia/environments/v1.7/Manifest.toml`
   Resolving package versions...
   Installed CodecZlib ────────── v0.7.0
   Installed FilePathsBase ────── v0.9.17
   Installed SentinelArrays ───── v1.3.12
   Installed WeakRefStrings ───── v1.4.1
   Installed InlineStrings ────── v1.1.2
   Installed TranscodingStreams ─ v0.9.6
   Installed CSV ──────────────── v0.10.2
    Updating `/opt/julia/environments/v1.7/Project.toml`
  [336ed68f] + CSV v0.10.2
    Updating `/opt/julia/environments/v1.7/Manifest.toml`
  [336ed68f] + CSV v0.10.2
  [944b1d66] + CodecZlib v0.7.0
  [48062228] + FilePathsBase v0.9.17
  [842dd82b] + InlineStrings v1.1.2
  [91c51154] + SentinelArrays v1.3.12
  [3bb67fe8] + TranscodingStreams v0.9.6
  [ea10d353] + WeakRefStrings v1.4.1
Precompiling project...
  ✓ TranscodingStreams
  ✓ InlineStrings
  ✓ CodecZlib
  ✓ SentinelArrays
  ✓ FilePathsBase
  ✓ WeakRefString

In [333]:
using Random
using DataFrames
using Statistics
using CSV
using TickTock

In [19]:
function get_instancia(n)
    return rand(1:100, n)
end

function get_permutacoes(instancia)
    return [shuffle(instancia) for _=1:20]
end

get_permutacoes (generic function with 1 method)

In [272]:
function get_custo(instancia, tamanho, custo_total)
    if tamanho == 1
        return instancia[tamanho], 0
    else
        espera, custo_total = get_custo(instancia[1:tamanho-1], tamanho-1, custo_total)
        custo = instancia[tamanho] + espera
        return custo, custo_total + custo
    end
end

function get_vizinhos_one_flip(solucao, tamanho)
    vizinhos = Vector()
    for indice in 1:tamanho-1
        vizinho = copy(solucao)
        vizinho[indice], vizinho[indice+1] = solucao[indice+1], solucao[indice]
        push!(vizinhos, vizinho)
    end
    vizinho = copy(solucao)
    vizinho[1], vizinho[tamanho] = solucao[tamanho], solucao[1]
    push!(vizinhos, vizinho)
    return vizinhos
end

get_vizinhos_one_flip (generic function with 2 methods)

In [264]:
function get_melhor_vizinho(vizinhos, is_first_improved, menor_custo, melhor_solucao, tamanho)
    for vizinho in vizinhos
        _, custo = get_custo(vizinho, tamanho, 0)
        if custo < menor_custo
            menor_custo = custo
            melhor_solucao = vizinho
            if is_first_improved
                break
            end
        end
    end
    return melhor_solucao, menor_custo
end

get_melhor_vizinho (generic function with 1 method)

In [325]:
resultados = []
for is_first_improved in [true, false]
    if is_first_improved
        alg = "FI"
    else
        alg = "BI"
    end
    for n in [100, 200, 300, 400, 500]
        tick()
        instancia_inicial = get_instancia(n)
        permutacoes_iniciais = get_permutacoes(instancia_inicial)
        solucao_inicial = Nothing
        menor_custo = 99999999999
        solucao_inicial, menor_custo = get_melhor_vizinho(permutacoes_iniciais, is_first_improved, 
                                        menor_custo, solucao_inicial, n)
        solucao = solucao_inicial
        for iteracao in 1:100
            vizinhos = get_vizinhos_one_flip(solucao, n)
            solucao, menor_custo = get_melhor_vizinho(vizinhos, is_first_improved, menor_custo, solucao, n)
        end
        tempo = tok()
        push!(resultados, tuple(alg, n, 20, round(tempo, digits=2), "NA", menor_custo))
    end
end

┌ Info:  started timer at: 2022-02-28T19:57:18.640
└ @ TickTock /opt/julia/packages/TickTock/RsTHR/src/TickTock.jl:32
┌ Info:  started timer at: 2022-02-28T19:57:18.682
└ @ TickTock /opt/julia/packages/TickTock/RsTHR/src/TickTock.jl:32
┌ Info:  started timer at: 2022-02-28T19:57:18.847
└ @ TickTock /opt/julia/packages/TickTock/RsTHR/src/TickTock.jl:32
┌ Info:  started timer at: 2022-02-28T19:57:19.124
└ @ TickTock /opt/julia/packages/TickTock/RsTHR/src/TickTock.jl:32
┌ Info:  started timer at: 2022-02-28T19:57:19.563
└ @ TickTock /opt/julia/packages/TickTock/RsTHR/src/TickTock.jl:32
┌ Info:  started timer at: 2022-02-28T19:57:20.059
└ @ TickTock /opt/julia/packages/TickTock/RsTHR/src/TickTock.jl:32
┌ Info:  started timer at: 2022-02-28T19:57:20.450
└ @ TickTock /opt/julia/packages/TickTock/RsTHR/src/TickTock.jl:32
┌ Info:  started timer at: 2022-02-28T19:57:23.062
└ @ TickTock /opt/julia/packages/TickTock/RsTHR/src/TickTock.jl:32
┌ Info:  started timer at: 2022-02-28T19:57:30.426
└ @ T

In [326]:
df = DataFrame(NamedTuple{(:alg, :n, :no, :time, :iterations, :value)}.(resultados))

,alg,n,no,time,iterations,value
,String,Int64,Int64,Float64,String,Int64
1,FI,100,20,0.04,NA,246750
2,FI,200,20,0.16,NA,989974
3,FI,300,20,0.28,NA,2502569
4,FI,400,20,0.44,NA,3942759
5,FI,500,20,0.5,NA,6027367
6,BI,100,20,0.39,NA,220849
7,BI,200,20,2.65,NA,990059
8,BI,300,20,7.36,NA,2159316
9,BI,400,20,12.41,NA,3969037


In [328]:
CSV.write("resultados_tarefa2.csv", df)

"resultados_tarefa2.csv"

### Tempo

#### Média

In [340]:
mean(df.time)

4.763

#### Desvio padrão

In [341]:
std(df.time)

7.721541368866136

### Custo

#### Média

In [342]:
mean(df.value)

2.723623e6

#### Desvio padrão

In [343]:
std(df.value)

2.2292931393013345e6